In [1]:
from vllm import LLM, SamplingParams

# Create an LLM
llm = LLM(
    model="Qwen/Qwen2.5-72B",
    gpu_memory_utilization=0.95,
    max_model_len=16384,
    tensor_parallel_size=4,
)

INFO 04-06 09:21:51 [__init__.py:239] Automatically detected platform cuda.
INFO 04-06 09:21:59 [config.py:585] This model supports multiple tasks: {'score', 'reward', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 04-06 09:21:59 [config.py:1519] Defaulting to use mp for distributed inference
INFO 04-06 09:21:59 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-06 09:22:00 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='Qwen/Qwen2.5-72B', speculative_config=None, tokenizer='Qwen/Qwen2.5-72B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfi

Loading safetensors checkpoint shards:   0% Completed | 0/37 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=119945) INFO 04-06 09:22:19 [loader.py:447] Loading weights took 12.29 seconds
(VllmWorker rank=2 pid=119982) INFO 04-06 09:22:19 [loader.py:447] Loading weights took 12.12 seconds
(VllmWorker rank=1 pid=119945) INFO 04-06 09:22:19 [gpu_model_runner.py:1186] Model loading took 34.0070 GB and 13.189715 seconds
(VllmWorker rank=2 pid=119982) INFO 04-06 09:22:19 [gpu_model_runner.py:1186] Model loading took 34.0070 GB and 13.311991 seconds
(VllmWorker rank=0 pid=119920) INFO 04-06 09:22:20 [loader.py:447] Loading weights took 12.69 seconds
(VllmWorker rank=0 pid=119920) INFO 04-06 09:22:20 [gpu_model_runner.py:1186] Model loading took 34.0070 GB and 14.515650 seconds
(VllmWorker rank=3 pid=120014) INFO 04-06 09:22:21 [loader.py:447] Loading weights took 13.70 seconds
(VllmWorker rank=3 pid=120014) INFO 04-06 09:22:21 [gpu_model_runner.py:1186] Model loading took 34.0070 GB and 15.201918 seconds
(VllmWorker rank=0 pid=119920) INFO 04-06 09:22:37 [backends.py:415] Usi

In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from my_parser import process_zips

In [3]:
ZIP_PATHS = [
    "../data/datathon_part1.zip",
    "../data/datathon_part2.zip",
    "../data/datathon_part3.zip",
    "../data/datathon_part4.zip",
]
all_clients = process_zips(ZIP_PATHS)

In [4]:
def set_keys_to_none(d):

    for key in d.keys():
        if isinstance(d[key], dict):
            set_keys_to_none(d[key])
        else:
            d[key] = ""
    return d

In [5]:
tmp = all_clients[4]["client_profile"].copy()
tmp = set_keys_to_none(tmp)

field_string = str(tmp)
field_string

"{'name': '', 'address': {'city': '', 'street name': '', 'street number': '', 'postal code': ''}, 'country_of_domicile': '', 'birth_date': '', 'nationality': '', 'passport_number': '', 'passport_issue_date': '', 'passport_expiry_date': '', 'gender': '', 'phone_number': '', 'email_address': '', 'marital_status': '', 'secondary_school': {'name': '', 'graduation_year': ''}, 'higher_education': '', 'employment_history': '', 'aum': {'savings': '', 'inheritance': '', 'real_estate_value': ''}, 'inheritance_details': {'relationship': '', 'inheritance year': '', 'profession': ''}, 'real_estate_details': '', 'investment_risk_profile': '', 'investment_horizon': '', 'investment_experience': '', 'type_of_mandate': '', 'preferred_markets': '', 'currency': ''}"

In [6]:
contraction_prompt = f"Does the description contain information that contradicts the client profile? If so, start your answer with YES, otherwise start your answer with NO."
conflict_prompt = f"Is there an obvious conflict of interest between the client and the bank? If so, start your answer with YES, otherwise start your answer with NO."

In [37]:
import pickle as pkl

false_positives = pkl.load(open("../false_positives.pkl", "rb"))[:400]
true_positives = pkl.load(open("../true_positives.pkl", "rb"))[:400]

considered_indices = false_positives + true_positives

In [ ]:
import datetime as dt


def get_client_description_prompt():
    return [
        (
            f"Here is the client profile {all_clients[i]['client_profile']}\n"
            f"Here is the description {all_clients[i]['client_description']}\n"
            "Does the description contradict the client profile net worth?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]


def get_age_prompt():
    birth_dates = [
        dt.datetime.strptime(c["client_profile"]["birth_date"], "%Y-%m-%d")
        for c in all_clients
    ]
    ages = [(dt.datetime.now() - birth_date).days // 365 for birth_date in birth_dates]

    return [
        (
            f"Here is the description {all_clients[i]['client_description']}\n"
            f"Is it true that the client is either {ages[i]-1} or {ages[i]} years old?"
            "If the text does not mention the age, say IDK."
            "If the text has the same age, say YES. If it does not have the same age, say NO."
            "Justify why after the answer."
        )
        for i in considered_indices
    ]


def get_marry_status_prompt():
    return [
        (
            f"Here is the client's marital status: <{all_clients[i]['client_profile']['marital_status']}>\n"
            f"Here is the Family Background of the client: <{all_clients[i]['client_description']['Family Background']}>\n"
            "Does the marital status match?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]

def get_education_prompt():
    return [
        (
            f"Here is the client's secondary school: <{all_clients[i]['client_profile']['secondary_school']}>\n"
            f"Here is the Education Background of the client: <{all_clients[i]['client_description']['Education Background']}>\n"
            "Does the secondary school match?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]

def get_higher_education_prompt():
    return [
        (
            f"Here is the client's higher education: {all_clients[i]['client_profile']['higher_education']}\n"
            f"Here is the Education Background of the client: {all_clients[i]['client_description']['Education Background']}\n"
            "Does the higher education match?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]

def get_inheritence_prompt():
    return [
        (
            f"Here is the inheritence of the client: <{all_clients[i]['client_profile']['aum']['inheritance']}>\n"
            #f"Here are the inheritence details of the client: <{all_clients[i]['client_profile']['inheritance_details']}>\n"
            f"Here is the Wealth Summary of the client: <{all_clients[i]['client_description']['Wealth Summary']}>\n"
            "Does the inheritence match?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]

def get_real_estate_value_prompt():
    return [
        (
            f"Here is the real-estate value of the client: <{all_clients[i]['client_profile']['aum']['real_estate_value']}>\n"
            f"Here is the Wealth Summary of the client: <{all_clients[i]['client_description']['Wealth Summary']}>\n"
            "Does the real-estate value match?\n"
            "If this is the case say YES, otherwise say NO."
        )
        for i in considered_indices
    ]

prompts = get_inheritence_prompt()

In [91]:
print(prompts[0])

Here is the inheritence of the client: <1620000>
Here is the Wealth Summary of the client: <While working, she saved 340000 EUR, which she used to build a diversified investment portfolio.
She owns house in Turin, where she resides. It is worth 1540000 EUR.

In addition to her primary residence, she owns villa in Siena worth 1740000 EUR.

She inherited a significant sum of 1620000 EUR from her grandmother, a respected Venture Capitalist, in 2006, which she has wisely invested.
>
Does the inheritence match?
If this is the case say YES, otherwise say NO.


In [92]:
# adjust sampling parameters as necessary for the task
sampling_params = SamplingParams(temperature=0.0, max_tokens=100, min_p=0.15, top_p=0.7)

# Generate texts from the prompts
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 800/800 [00:04<00:00, 198.15it/s, est. speed input: 27863.70 toks/s, output: 396.32 toks/s]


In [93]:
answer = [o.outputs[0].text for o in outputs]

In [94]:
len(answer)

800

In [95]:
false_pos_rej = [
    (i, a)
    for i, a in zip(considered_indices, answer)
    if a.startswith(" YES") and i in false_positives
]

len(false_pos_rej)

45

In [96]:
false_pos_rej

[(10, ' YES'),
 (151, ' YES'),
 (183, ' YES'),
 (349, ' YES'),
 (370, ' YES'),
 (578, ' YES'),
 (690, ' YES'),
 (713, ' YES'),
 (758, ' YES'),
 (775, ' YES'),
 (932, ' YES'),
 (957, ' YES'),
 (976, ' YES'),
 (990, ' YES'),
 (1060, ' YES'),
 (1120, ' YES'),
 (1253, ' YES'),
 (1280, ' YES'),
 (1471, ' YES'),
 (1491, ' YES'),
 (1585, ' YES'),
 (1686, ' YES'),
 (1970, ' YES'),
 (2036, ' YES'),
 (2119, ' YES'),
 (2191, ' YES'),
 (2237, ' YES'),
 (2259, ' YES'),
 (2310, ' YES'),
 (2362, ' YES'),
 (2388, ' YES'),
 (2563, ' YES'),
 (2657, ' YES'),
 (2707, ' YES'),
 (2751, ' YES'),
 (2800, ' YES'),
 (2863, ' YES'),
 (2865, ' YES'),
 (2966, ' YES'),
 (3012, ' YES'),
 (3056, ' YES'),
 (3202, ' YES'),
 (3222, ' YES'),
 (3245, ' YES'),
 (3400, ' YES')]

In [97]:
true_pos_rej = [
    (i, a)
    for i, a in zip(considered_indices, answer)
    if a.startswith(" YES") and i in true_positives
]

len(true_pos_rej)

54

In [98]:
true_pos_rej

[(0, ' YES'),
 (65, ' YES'),
 (67, ' YES'),
 (76, ' YES'),
 (82, ' YES'),
 (85, ' YES'),
 (88, ' YES'),
 (96, ' YES'),
 (98, ' YES'),
 (119, ' YES'),
 (121, ' YES'),
 (131, ' YES'),
 (168, ' YES'),
 (179, ' YES'),
 (186, ' YES'),
 (223, ' YES'),
 (253, ' YES'),
 (279, ' YES'),
 (282, ' YES'),
 (284, ' YES'),
 (300, ' YES'),
 (320, ' YES'),
 (328, ' YES'),
 (331, ' YES'),
 (335, ' YES'),
 (371, ' YES'),
 (375, ' YES'),
 (376, ' YES'),
 (383, ' YES'),
 (400, ' YES'),
 (415, ' YES'),
 (420, ' YES'),
 (449, ' YES'),
 (452, ' YES'),
 (516, ' YES'),
 (529, ' YES'),
 (543, ' YES'),
 (576, ' YES'),
 (614, ' YES'),
 (625, ' YES'),
 (637, ' YES'),
 (647, ' YES'),
 (680, ' YES'),
 (716, ' YES'),
 (737, ' YES'),
 (762, ' YES'),
 (769, ' YES'),
 (781, ' YES'),
 (787, ' YES'),
 (792, ' YES'),
 (795, ' YES'),
 (802, ' YES'),
 (806, ' YES'),
 (831, ' YES')]

In [ ]:
all_clients[61]

{'passport': {'first_name': 'Julia',
  'middle_name': 'Fleur',
  'last_name': 'Schipper',
  'gender': 'F',
  'country': 'Netherlands',
  'country_code': 'NLD',
  'nationality': 'Dutch',
  'birth_date': '1971-05-21',
  'passport_number': 'BF3878404',
  'passport_mrz': ['P<NLDSCHIPPER<<JULIA<FLEUR<<<<<<<<<<<<<<<<<<<',
   'BF3878404NLD710521<<<<<<<<<<<<<<<<<<<<<<<<<<<'],
  'passport_issue_date': '2023-07-04',
  'passport_expiry_date': '2033-07-03'},
 'client_profile': {'name': 'Julia Fleur Schipper',
  'address': {'city': 'Haarlem',
   'street name': 'Maliebaan',
   'street number': 84,
   'postal code': '1203 12'},
  'country_of_domicile': 'Netherlands',
  'birth_date': '1971-05-21',
  'nationality': 'Dutch',
  'passport_number': 'BF3878404',
  'passport_issue_date': '2023-07-04',
  'passport_expiry_date': '2033-07-03',
  'gender': 'F',
  'phone_number': '+31 06 93346627',
  'email_address': 'julia.schipper@icloud.com',
  'marital_status': 'widowed',
  'secondary_school': {'name': 'Openb